<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.883 · Aprendizaje por refuerzo</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Ejemplo: Método de MonteCarlo en el entorno WindyGridWorld

En este ejemplo implementaremos el método de MonteCarlo de aprendizaje por refuerzo para buscar una solución óptima en el problema de WindyGridWorld.

## 1. El entorno __WindyGridWorld__

El entorno __WindyGridWorld__ consiste en un agente que se mueve en una cuadrícula 7x10 (alto x ancho). En cada paso, el agente tiene 4 opciones de acción o movimiento: ARRIBA, ABAJO, DERECHA, IZQUIERDA. El agente siempre sale de la misma casilla [3, 0] y el juego termina cuando el agente llega a la casilla de llegada [3, 7]. 

El entorno se corresponde con el ejemplo 'Cuadrícula con viento' explicado en la sección 3.1.2. el módulo "Métodos de Diferencia Temporal". El problema radica en que hay un viento que empuja al agente hacia arriba en la parte central de la cuadrícula. Esto provoca que, aunque se ejecute una acción estándar, en la región central los estados resultantes se desplazan hacia arriba por un viento cuya fuerza varía entre columnas.

<img src="../figs/GridWorld.png">

El código para implementar este entorno, que se encuentra disponible en el fichero adjunto `windy_gridworld_env.py`, ha sido adaptado del siguiente enlace:

https://pypi.org/project/gym-gridworlds/

## 2. Métodos de Montecarlo

Dado que el entorno es determinista, es factible encontrar una política óptima (que puede no ser única) que consiga el mayor retorno (y por tanto la trayectoria más corta).

El objetivo de este apartado es realizar una estimación de la política óptima mediante los métodos de Montecarlo, en concreto estudiaremos el algoritmo *On-policy first-visit MC control (para políticas $\epsilon$-soft)*.

A continuación, implementaremos el Algoritmo 3 explicado en el módulo "Métodos de Montecarlo" utilizando los siguientes parámetros:
    
- Número de episodios = 250000
- Epsilon = 0.1
- Factor de descuento = 1

In [1]:
## Solución adaptada de: https://github.com/dennybritz/reinforcement-learning/tree/master/MC

import gymnasium as gym
import numpy as np
from collections import defaultdict
import sys
import windy_gridworld_env as wge

#Cargamos el entrono
env = wge.WindyGridworldEnv()

def make_epsilon_greedy_policy(Q, epsilon, num_Actions):
    """
    Crea una política epsilon-greedy basada en una función de valor de acción Q y epsilon
    
    Args:
        Q: Un diccionario cuya correspondencia es state -> action-values.
           Cada valor es un array de numpy de longitud num_Actions (see below)
        epsilon: La probabilidad de seleccionar una acción aleatoria (float entre 0 and 1).
        num_Actions: Número de acciones del entorno. (en el caso del WIndyGridWorld es 4)
    
    Returns:
        Una función que tome como argumento la observación y devuelva como resultado
        las probabilidades de cada acción como un array de numpy de longitud num_Actions.
    """
    def policy_fn(observation):

        A = np.ones(num_Actions, dtype=float) * epsilon / num_Actions
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)

        return A
    
    return policy_fn

def mc_control_on_policy_epsilon_greedy(env, num_episodes, discount=1.0, epsilon=0.1):
    """
    Control mediante métodos de Montecarlo usando políticas Epsilon-Greedy
    Encuentra una política epsilon-greedy.
    
    Args:
        env: entorno OpenAI gym.
        num_episodes: Número de episodios de la muestra.
        discount: factor de descuento.
        epsilon: La probabilidad de seleccionar una acción aleatoria (float entre 0 and 1)
    
    Returns:
        Una tupla (Q, policy).
        Q: Un diccionario cuya correspondencia es state -> action-values.
        policy: Una función que toma como argumento la observación y devuelve como resultado
                las probabilidades de cada acción
    """
    
    # Almacenamos la suma y el número de retornos de cada estado para calcular
    # el promedio. Podríamos usar un array para guardar todos los retornos
    # (como en el libro) pero es ineficiente en términos de memoria.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # La función de valor de acción Q.
    # Un diccionario anidado cuya correspondencia es state -> (action -> action-value).
    # Inicialmente la inicializamos a cero
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # La política que estamos siguiendo
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
    for i_episode in range(1, num_episodes + 1):
        # Imprimimos en qué episodio estamos, útil para debugar.
        if i_episode % 10 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()

        # Generamos un episodio y lo almacenamos
        # Un episodio es un array de las tuplas (state, action, reward)
        episode = []
        state, info = env.reset()
        done = False
        for t in range(1000): # limitamos el número de time-steps de cada episodio a 1000
            probs = policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode.append((state, action, reward))
            if done:
                break
            state = next_state

        # Encontramos todos los pares (estado, acción) que hemos visitado en este episodio
        # Convertimos cada estado en una tupla para poder usarlo como clave del diccionario
        sa_in_episode = set([(tuple(x[0]), x[1]) for x in episode])
        for state, action in sa_in_episode:
            sa_pair = (state, action)
            # Encontramos la primera aparición del par (estado, acción) en el episodio
            first_occurence_idx = next(i for i,x in enumerate(episode)
                                       if x[0] == state and x[1] == action)
            # Sumamos todas las recompensas desde la primera aparición
            G = sum([x[2]*(discount**i) for i,x in enumerate(episode[first_occurence_idx:])])
            # Calculamos el retorno promedio para este estado en todos los episodios muestreados
            returns_sum[sa_pair] += G
            returns_count[sa_pair] += 1.0
            Q[state][action] = returns_sum[sa_pair] / returns_count[sa_pair]
        
        # La política se mejora implícitamente al ir cambiando los valores de Q
    
    return Q, policy

In [2]:
Q, policy = mc_control_on_policy_epsilon_greedy(env, num_episodes=250000, discount=1, epsilon=0.1)

Episode 250000/250000.

Seguidamente, implementaremos una función que imprima por pantalla la política óptima encontrada para cada celda. 

In [3]:
def print_policy(policy, height, width):

    switch_action = {
        0: "U",
        1: "R",
        2: "D",
        3: "L"
    }
    for i in range(height):
        print("------------------------------------------------------------------------------------------")
        for j in range(width):
            arr = np.array(policy((i,j)))
            act = int(np.where(arr == np.amax(arr))[0])
            a = switch_action[act]
            print("  %s  |" % a, end="")
        print("")

In [4]:
print_policy(policy,7,10)

------------------------------------------------------------------------------------------
  L  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  R  |  D  |
------------------------------------------------------------------------------------------
  D  |  L  |  U  |  L  |  R  |  R  |  L  |  U  |  L  |  D  |
------------------------------------------------------------------------------------------
  R  |  R  |  R  |  R  |  U  |  R  |  R  |  R  |  R  |  D  |
------------------------------------------------------------------------------------------
  U  |  L  |  R  |  U  |  D  |  R  |  D  |  U  |  R  |  D  |
------------------------------------------------------------------------------------------
  L  |  U  |  R  |  L  |  R  |  D  |  U  |  D  |  L  |  L  |
------------------------------------------------------------------------------------------
  R  |  D  |  R  |  R  |  L  |  U  |  U  |  D  |  L  |  L  |
----------------------------------------------------------------------------------------

Ejecutamos un episodio con la política óptima encontrada y mostramos la trayectoria del agente y el retorno obtenido.

In [5]:
def execute_episode_MC(policy, env):
    obs, info = env.reset()
    t, total_reward, done = 0, 0, False
    print("Obs inicial: {} ".format(obs))

    switch_action = {
            0: "U",
            1: "R",
            2: "D",
            3: "L",
        }

    for t in range(1000): # limitamos el número de time-steps de cada episodio a 1000
        
        # Elegimos la política óptima en cada caso (el máximo de la política Epsilon-Greedy)
        arr = np.array(policy(obs))
        action = arr.argmax()
    
        # Ejecutamos la acción y esperamos la respuesta del entorno
        new_obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        obs = new_obs
        print("Action: {} -> Obs: {} and reward: {}".format(switch_action[action], obs, reward))

        if t==999:
            print("Number of time-septs exceeds 1000. STOP episode.")
            
        total_reward += reward
        t += 1
        if done:
            break
        
    print("Episode finished after {} timesteps and reward was {} ".format(t, total_reward))
    env.close()

In [6]:
execute_episode_MC(policy,env)

Obs inicial: (3, 0) 
Action: U -> Obs: (2, 0) and reward: -1
Action: R -> Obs: (2, 1) and reward: -1
Action: R -> Obs: (2, 2) and reward: -1
Action: R -> Obs: (2, 3) and reward: -1
Action: R -> Obs: (1, 4) and reward: -1
Action: R -> Obs: (0, 5) and reward: -1
Action: R -> Obs: (0, 6) and reward: -1
Action: R -> Obs: (0, 7) and reward: -1
Action: R -> Obs: (0, 8) and reward: -1
Action: R -> Obs: (0, 9) and reward: -1
Action: D -> Obs: (1, 9) and reward: -1
Action: D -> Obs: (2, 9) and reward: -1
Action: D -> Obs: (3, 9) and reward: -1
Action: D -> Obs: (4, 9) and reward: -1
Action: L -> Obs: (4, 8) and reward: -1
Action: L -> Obs: (3, 7) and reward: -1
Episode finished after 16 timesteps and reward was -16 
